# Tutorial

### Thermodynamic and material properties are handled by Chemical objects

Chemical objects are an extension of the [thermo.Chemical](http://thermo.readthedocs.io/en/latest/thermo.chemical.html) class from the Chemical Engineering Design Library.

Initiallize a Chemical object with an ID:

In [6]:
from biosteam import Chemical
Water = Chemical('Water')
Water

<Chemical: Water>

Chemical objects have a temperature (T), pressure (P), and phase

In [7]:
Water.T, Water.P, Water.phase

(298.15, 101325, 'l')

All biosteam objects have a `show` method

In [8]:
Water.show()

Chemical: Water
 phase: 'l', T: 298.15 K, P: 101325 Pa


You can change the units for `show`

In [9]:
Water.show(T='degC', P='atm')

Chemical: Water
 phase: 'l', T: 25 degC, P: 1 atm


Chemical objects contain thermodynamic properties at the conditions:

In [10]:
Water.rho # (kg/m3)

997.0247522552814

These properties are dependent on temperature (T), pressure (P) and phase:

In [5]:
Water.T = 350 # (Kelvin)
Water.rho # (kg/m3)

971.4427816206321

`Note how the density changed with temperature`

Many more material properties are available. Please read the [thermo.Chemical](http://thermo.readthedocs.io/en/latest/thermo.chemical.html) documentation to learn more.

### Chemical objects are managed by a Species object

Initiallize a Species object with IDs:

In [7]:
from biosteam import Species
species = Species('Methanol', 'Glycerol')
species

<Species: (Methanol, Glycerol)>

The Chemical objects are stored as attributes:

In [8]:
species.Methanol, species.Glycerol

(<Chemical: Methanol>, <Chemical: Glycerol>)

Set chemical attributes:

In [9]:
species.Water = Water
species

<Species: (Methanol, Glycerol, Water)>

### Material flows are handled by the Stream object

First set the working species of all Stream objects:

In [10]:
from biosteam import Stream
Stream.species = species # From before

A Stream is initialized with an ID, flow rates, temperature, pressure and phase:

In [11]:
feed1 = Stream(ID='feed1', flow=[1,2,3], units='kg/s', T=300, P=101325)
feed1.show(flow='kg/hr') # Change flow units for show

Stream: feed1
 phase: 'l', T: 300 K, P: 101325 Pa
 flow (kg/hr): Methanol  3.6e+03
               Glycerol  7.2e+03
               Water     1.08e+04


Alternatively, you can provide specie-flow rate pairs and leave the rest default:

In [12]:
feed2 = Stream(ID='feed2', Water=1000, Glycerol=200)
feed2.show()

Stream: feed2
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Glycerol  200
                 Water     1e+03


Stream objects contain T, P, and phase dependent properties:

In [13]:
feed1.H # kJ/hr with reference at STP

132066.06431094545

In [14]:
feed2.H

0.0

### Unit processes are found in the Units subpackage

Units are initialized with an ID, and unit specific arguments:

In [15]:
from biosteam.units import Flash
F1 = Flash('F1', V=0.5, P=101325) # Specify vapor fraction and isobaric conditions

Set input streams 'ins':

In [16]:
F1.ins = feed1
F1.show()

Flash: F1
ins...
[0] feed1
    phase: 'l', T: 300 K, P: 101325 Pa
    flow (kmol/hr): Methanol  112
                    Glycerol  78.2
                    Water     599
outs...
[0] d1
    phase: 'g', T: 298.15 K, P: 101325 Pa
    flows:  0
[1] d2
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flows:  0


To simulate the flash, use the 'simulate' method:

In [17]:
F1.simulate()
F1.show()

Flash: F1
ins...
[0] feed1
    phase: 'l', T: 300 K, P: 101325 Pa
    flow (kmol/hr): Methanol  112
                    Glycerol  78.2
                    Water     599
outs...
[0] d1
    phase: 'g', T: 368.5 K, P: 101325 Pa
    flow (kmol/hr): Methanol  85.5
                    Glycerol  0.0332
                    Water     309
[1] d2
    phase: 'l', T: 368.5 K, P: 101325 Pa
    flow (kmol/hr): Methanol  26.8
                    Glycerol  78.1
                    Water     290


All the results are in the `results` SmartBook object `SmartBook <https://bookkeep.readthedocs.io/en/latest/SmartBook.html>`__

In [18]:
F1.results

{'Operation': {},
 'Design': 
    {'SepType': Vertical,
     'Length': 13.5 (ft),
     'Diameter': 6.5 (ft),
     'Weight': 5.31e+03 (lb),
     'Wall thickness': 0.375 (in)},
 'Cost': 
    {'Vessel': 4.3e+04 (USD),
     'Heat exchanger': 2.21e+04 (USD)}}

### Recycle loops and process specifications are handled by System objects

Create a Mixer object and a Splitter object:

In [19]:
from biosteam.units import Mixer, Splitter
M1 = Mixer('M1')
S1 = Splitter('S1', split=0.5) # Split to 0th output stream

Connect streams and make a recycle loop using "Pipe" notation:

In [20]:
[S1-0, fresh]-M1    # M1.ins = [S1.outs[0], fresh]
M1-F1             # F1.ins = M1.outs
F1-1-S1           # S1.ins = F1.outs[1]

NameError: name 'fresh' is not defined

Create System object by specifying an ID, a recycle stream and a network, 'net', of units:

In [ ]:
from biosteam import System
sys1 = System('sys1', network=(M1, F1, S1), recycle=S1-0) # recycle=S1.outs[0]
sys1.show()

View the System object as a Graphviz diagram:

In [ ]:
sys1.diagram

Converge the System object:

In [ ]:
sys1.converge()
sys1.show()

In [ ]:
S1.outs[0].show()